## 0. Install and Import Modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [ ]:
# import os
# import sys
# import torch
# need_pytorch3d=False
# try:
#     import pytorch3d
# except ModuleNotFoundError:
#     need_pytorch3d=True
# if need_pytorch3d:
#     if torch.__version__.startswith("1.13.") and sys.platform.startswith("linux"):
#         # We try to install PyTorch3D via a released wheel.
#         pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
#         version_str="".join([
#             f"py3{sys.version_info.minor}_cu",
#             torch.version.cuda.replace(".",""),
#             f"_pyt{pyt_version_str}"
#         ])
#         !pip install fvcore iopath
#         !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
#     else:
#         # We try to install PyTorch3D from source.
#         !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
#         !tar xzf 1.10.0.tar.gz
#         os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
#         !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=2a5a735959b1ed1a5c444b3f0a8b8c4fc0d7f2c5ffc6c6c47b9fce8c6fd6784d
  Stored in directory: /root/.cache/pip/wheels/b8/79/07/c0e9367f5b5ea325e246bd73651e8af175fabbef943043b1cc
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=201007029374099283742884f8d86469d1265b3240302b52f894036335fd8d5b
  Stored in directory: /root/.cache/pip/wheels/89/3e/24/0f349c0b2eeb6965903035f3b00dbb5c9bea437b4a2f18d82c
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu116_pyt1131/dow

In [ ]:
# # imports
# import numpy as np
# import torch
# from pytorch3d.transforms.so3 import (
#     so3_exp_map,
#     so3_relative_angle,
# )
# from pytorch3d.transforms.se3 import (
#     se3_exp_map,
#     se3_log_map,
# )

# import pytorch3d.transforms as transforms
    
# # add path for demo utils
# import sys
# import os
# sys.path.append(os.path.abspath(''))

# # set for reproducibility
# torch.manual_seed(42)
# if torch.cuda.is_available():
#     device = torch.device("cuda:0")
# else:
#     device = torch.device("cpu")
#     print("WARNING: CPU only, this will be slow!")

## **Data preparation**

In [ ]:
# # Load the matrices 
# T_o_cb_batch = np.load('')
# batch_size = T_o_cb_batch.shape[0]
# T_o_p_batch = np.load('')
# T_c_p_batch = np.load('')
# X_matrix = np.load('')
# X_batch = np.tile(X_matrix, [batch_size,1,1])

# # Load into torch tensor
# X_batch = torch.tensor(X_batch)
# T_o_p_batch = torch.tensor(T_o_p_batch)
# T_o_cb_batch = torch.tensor(T_o_cb_batch)
# T_c_p_batch = torch.tensor(T_c_p_batch)

In [ ]:
# ## test
# from scipy.spatial.transform import Rotation as R

# batch_size = 10
# T_o_cb_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_o_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_c_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# X_matrix = np.hstack((np.vstack((R.random().as_matrix(), np.zeros([1,3]))), np.ones([4,1])))
# X_batch = np.tile(X_matrix, [batch_size,1,1])
# X_batch[:, :3, 3] = 2
# print(X_batch[1, :,:])
# X_batch = torch.tensor(X_batch)
# T_o_p_batch = torch.tensor(T_o_p_batch)
# T_o_cb_batch = torch.tensor(T_o_cb_batch)
# T_c_p_batch = torch.tensor(T_c_p_batch)

[[-0.49508795 -0.8619929  -0.10888599  2.        ]
 [-0.86414445  0.47551297  0.16474763  2.        ]
 [-0.09023459  0.17565779 -0.98030712  2.        ]
 [ 0.          0.          0.          1.        ]]


In [ ]:
# # Create Transform3d objects from the batch of transformation matrices
# T_o_cb = transforms.Transform3d(matrix=T_o_cb_batch, device=device)
# X = transforms.Transform3d(matrix=X_batch, device=device)
# T_o_p = transforms.Transform3d(matrix=T_o_p_batch, device=device)
# T_c_p = transforms.Transform3d(matrix=T_c_p_batch, device=device)

# # cam_gt: the ground truth camera pose w.r.t the optical tracker cam_gt = T_o_p * (T_c_p)^-1
# cam_gt = T_o_cb.compose(X)

# # cam_real: the camera pose from initial hand-eye calibration cam_real = T_o_cb * X
# cam_real = T_o_p.compose(T_c_p.inverse())

## **Define optimization loss function**



In [ ]:
# def camera_distance(hand_eye_real, hand_eye_gt):
#   '''
#   Calculate the l2 distance in se(3) manifold for the camera.
#   distance = sum||log(hand_eye_real*hand_eye_gt)||_2
#   '''

#   # Convert a batch of 4x4 transformation matrices transform to 
#   # a batch of 6-dimensional SE(3) logarithms of the SE(3) matrices
#   # print(hand_eye_real.compose(hand_eye_gt.inverse()).get_matrix())
#   tmp_matrix = hand_eye_real.compose(hand_eye_gt.inverse()).get_matrix()
#   vec = se3_log_map(tmp_matrix.permute(0,2,1)).sum(0)
#   # print(torch.norm(vec))

#   return torch.norm(vec)
  

# # camera_distance(cam_real, cam_gt)

## **Optimization**

In [ ]:
# # Create Transform3d objects from the batch of transformation matrices
# T_o_cb = transforms.Transform3d(matrix=T_o_cb_batch, device=device)
# X = transforms.Transform3d(matrix=X_batch, device=device)
# T_o_p = transforms.Transform3d(matrix=T_o_p_batch, device=device)
# T_c_p = transforms.Transform3d(matrix=T_c_p_batch, device=device)

# # hand_eye_real: the hand-eye transformation from initial calibration X
# hand_eye_real_6D = se3_log_map(X.get_matrix().permute(0, 2, 1))
# hand_eye_real_6D.requires_grad = True


# # hand_eye_gt: the hand-eye transformation from the chain 
# # hand_eye_gt = (T_o_cb)^-1 * T_o_p * (T_c_p)^-1
# hand_eye_gt = T_o_cb.inverse().compose(T_o_p).compose(T_c_p.inverse())

# # init the optimizer
# optimizer = torch.optim.SGD([hand_eye_real_6D], lr=.001, momentum=0.9)

# # run the optimization
# n_iter = 2000  # fix the number of iterations
# for it in range(n_iter):
#     # re-init the optimizer gradients
#     optimizer.zero_grad()

#     # transform the 6d vector into 4x4 matrix
#     hand_eye_real = se3_exp_map(hand_eye_real_6D).permute(0, 2, 1)
#     hand_eye_real = transforms.Transform3d(matrix=hand_eye_real, device=device)

#     # compare the composed cameras with the ground truth relative cameras
#     # camera_distance corresponds to $d$ from the description
#     loss = \
#         camera_distance(hand_eye_real, hand_eye_gt)
#     # loss.requires_grad_(True)

#     # our loss function is the camera_distance
#     loss.backward()
    
#     # apply the gradients
#     optimizer.step()

#     # # plot and print status message
#     if it % 200==0 or it==n_iter-1:
#         status = 'iteration=%3d; camera_distance=%1.3e' % (it, loss)
#         print(status)
        
# print('Optimization finished.')

iteration=  0; camera_distance=4.696e+01
iteration=200; camera_distance=2.786e+01
iteration=400; camera_distance=7.859e+00
iteration=600; camera_distance=5.260e-04
iteration=800; camera_distance=1.630e-02
iteration=1000; camera_distance=7.543e-03
iteration=1200; camera_distance=1.241e-02
iteration=1400; camera_distance=7.818e-03
iteration=1600; camera_distance=1.311e-02
iteration=1800; camera_distance=3.938e-03
iteration=1999; camera_distance=4.000e-03
Optimization finished.


## **Pypose implementation**

In [ ]:
# !pip install pypose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 KB 10.9 MB/s eta 0:00:00


In [1]:
import os
import sys
import numpy as np
import torch, pypose as pp
from torch import nn
from scipy.spatial.transform import Rotation as R

/home/shc/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

## **Data Preparation**

In [5]:
T_c_p_batch = np.load(os.path.join(params_save_dir, 'T_c_p_opti3.npy'))
T_c_p_batch_zed = np.load(os.path.join(params_save_dir, 'T_c_p_opti3zed.npy'))

T_c_p = pp.mat2SE3(T_c_p_batch)
T_c_p_zed = pp.mat2SE3(T_c_p_batch_zed)
# T_c_p @ pp.Inv(T_c_p_zed)
print([T_c_p[i][:3].norm().item() for i in range(160)],'\n',[T_c_p_zed[i][:3].norm().item() for i in range(41)])
# print(T_c_p_batch[0],'\n',T_c_p_batch_zed[0])

[287.46306079540096, 289.37015271170065, 290.8742758633388, 292.73261076181086, 293.5796410093682, 294.7381935021377, 295.65163780371245, 296.6717207639355, 296.60887942794585, 296.7720028680565, 295.78322598560777, 294.1113163891754, 292.479579375307, 291.13736352193337, 290.52611881555896, 290.49220670728005, 291.45006329538967, 292.3557645774705, 292.2122654252121, 294.00648728051505, 292.4641933640531, 293.6931333478941, 293.9162200572097, 293.8549692588349, 289.8480858806118, 287.74823788694033, 279.21396446518185, 269.8282117358135, 265.6479528302285, 255.79443880856488, 254.55568201187222, 252.22258230321665, 250.25050542213472, 244.66139490645753, 242.21454378241225, 240.05760053784599, 236.59552692660392, 235.08694196635162, 232.2532550097689, 229.77869754359227, 228.94334119619487, 226.73189780526943, 222.70302955661523, 217.71405267746792, 214.44398806207428, 211.72932159951156, 209.96437912494113, 207.25725169048124, 207.27289842786962, 203.53065319071774, 200.5994067591956

In [337]:
# # Load the matrices
params_save_dir = '../data/post_optimization_data/'
T_o_cb_batch = np.load(os.path.join(params_save_dir, 'T_o_cb_opti3.npy'))[:40]
batch_size = T_o_cb_batch.shape[0]
T_o_p_batch = np.load(os.path.join(params_save_dir, 'T_o_p_opti3.npy'))[:40]
T_c_p_batch = np.load(os.path.join(params_save_dir, 'T_c_p_opti3zed.npy'))[:40]
X_matrix = np.load('../params/hand_eye_X_0220.npy')
# X_batch = np.tile(X_matrix, [batch_size,1,1])[:40]

# convert to pp tensor
X = pp.mat2SE3(X_matrix)
T_o_p = pp.mat2SE3(T_o_p_batch)
T_o_cb = pp.mat2SE3(T_o_cb_batch)
T_c_p = pp.mat2SE3(T_c_p_batch)

# print(np.linalg.norm(X_batch[1, :3,3]))
print(X[:3].norm())

tensor(124.2816, dtype=torch.float64)


In [316]:
# # test
# batch_size = 800
# T_o_cb_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_o_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_c_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# X_matrix = np.hstack((np.vstack((R.random().as_matrix(), np.zeros([1,3]))), np.ones([4,1])))
# X_batch = np.tile(X_matrix, [batch_size,1,1])
# X_batch[:, :3, 3] = 2.5

# print(X_batch[1, :,:])

# # convert to pp tensor
# X = pp.mat2SE3(X_batch)
# T_o_p = pp.mat2SE3(T_o_p_batch)
# T_o_cb = pp.mat2SE3(T_o_cb_batch)
# T_c_p = pp.mat2SE3(T_c_p_batch)
# # print(X_batch[1,...])
# # print(X_batch @ pp.Inv(X_batch))

## **Optimization**

In [330]:
class CamDis(nn.Module):
    def __init__(self, hand_eye_real_6D):
        super().__init__()
        self.real = pp.Parameter(hand_eye_real_6D)

    def forward(self, input):
        # input hand_eye_gt
        hand_eye_real = self.real.Exp()
        tmp_quats = pp.Inv(hand_eye_real) @ input
        vec = tmp_quats.Log()
        # print(tmp_quats[0,:3])
        t = tmp_quats[:, :3].mean(0)/1000
        rot_vec = vec[:, 3:].mean(0)
        return torch.norm(rot_vec) + torch.norm(t)

In [338]:
hand_eye_gt = pp.Inv(T_o_cb) @ T_o_p @ pp.Inv(T_c_p)
hand_eye_gt = hand_eye_gt.to('cpu')
hand_eye_real = X
# hand_eye_gt[:3] = hand_eye_gt[:3]/1000
print(hand_eye_gt.shape)
print(X[:3].norm())
print([hand_eye_gt[i,:3].norm().item() for i in range(40)])
print(pp.Inv(hand_eye_real)@hand_eye_gt[14])
print(hand_eye_real@pp.Inv(hand_eye_gt[14]))


torch.Size([40, 7])
tensor(124.2816, dtype=torch.float64)
[129.38696244371238, 130.31321266780967, 131.22056046503752, 130.74747792270733, 130.9355447165963, 131.28286024867333, 131.71319895265736, 132.6399798819794, 134.09888508829454, 132.12391929493606, 132.32468656156016, 131.0044475015824, 131.47318927977182, 131.74028256573393, 131.0306383172398, 131.03875688633735, 132.98470118556992, 135.0633595663894, 142.25647029408242, 144.32143905572582, 127.45192506317801, 128.32031163174082, 129.29842102438286, 131.52221597077914, 126.57259807956595, 129.78327693911316, 133.75376028836806, 132.98695228159113, 133.69253823686347, 129.79472946272367, 131.03521025844276, 130.19661757430976, 130.79871061901835, 130.30468638285464, 130.56573388299273, 129.27139947572192, 129.59650309009226, 128.0953464916919, 126.90133237520305, 128.12610845788862]
SE3Type LieTensor:
LieTensor([ 2.1769e+00,  6.1200e+00,  6.1419e+00,  9.9980e-01, -1.9469e-02,
           -9.0935e-04,  4.1359e-03], dtype=torch.fl

In [340]:
# hand_eye_real: the hand-eye transformation from initial calibration X

hand_eye_real = X.clone().detach()
hand_eye_real_6D = hand_eye_real.Log()

# hand_eye_gt: the hand-eye transformation from the chain 
# hand_eye_gt = (T_o_cb)^-1 * T_o_p * (T_c_p)^-1
hand_eye_gt = pp.Inv(T_o_cb) @ T_o_p @ pp.Inv(T_c_p)
hand_eye_gt = hand_eye_gt.clone().detach()
# print(np.linalg.norm(hand_eye_gt[4].matrix()[:3,3]))
# print(np.linalg.norm(X[0].matrix()[:3,3]))

# init the optimizer
camera_distance = CamDis(hand_eye_real_6D).to(device)
input = hand_eye_gt.to(device)
strategy = pp.optim.strategy.Adaptive(damping=1e-6)
optimizer = pp.optim.LM(camera_distance, strategy=strategy)
# run the optimization
n_iter = 500  # fix the number of iterations
for it in range(n_iter):
    loss = optimizer.step(input)
    # # plot and print status message
    if it % 5==0 or it==n_iter-1:
        status = 'iteration=%3d; camera_distance=%1.3e' % (it, loss)
        print(status)
        # print(hand_eye_real_6D.Exp())
        print(input[0])
    if loss < 1e-6:
        print('Early Stopping with loss:', loss.item())
        break
# print(pp.matrix(hand_eye_real_6D[0].Exp()))
print('Optimization finished.')


iteration=  0; camera_distance=6.522e+00
SE3Type LieTensor:
LieTensor([ 25.2600, 106.1567,  69.5246,   0.7311,   0.3680,  -0.4909,   0.2985],
          device='cuda:0', dtype=torch.float64)
iteration=  5; camera_distance=2.858e+00
SE3Type LieTensor:
LieTensor([ 25.2600, 106.1567,  69.5246,   0.7311,   0.3680,  -0.4909,   0.2985],
          device='cuda:0', dtype=torch.float64)
iteration= 10; camera_distance=2.697e+00
SE3Type LieTensor:
LieTensor([ 25.2600, 106.1567,  69.5246,   0.7311,   0.3680,  -0.4909,   0.2985],
          device='cuda:0', dtype=torch.float64)
iteration= 15; camera_distance=2.473e+00
SE3Type LieTensor:
LieTensor([ 25.2600, 106.1567,  69.5246,   0.7311,   0.3680,  -0.4909,   0.2985],
          device='cuda:0', dtype=torch.float64)
iteration= 20; camera_distance=2.373e+00
SE3Type LieTensor:
LieTensor([ 25.2600, 106.1567,  69.5246,   0.7311,   0.3680,  -0.4909,   0.2985],
          device='cuda:0', dtype=torch.float64)
iteration= 25; camera_distance=2.366e+00
SE3Type L

In [335]:
res = hand_eye_real_6D.Exp()
X = X.to(device)
# print(res, X[0]@pp.Inv(res))
print(input[0,:3].norm())
print(X[:3].norm())
print(hand_eye_real_6D.Exp()[:3].norm())


tensor(129.3870, device='cuda:0', dtype=torch.float64)
tensor(124.2816, device='cuda:0', dtype=torch.float64)
tensor(50.0823, device='cuda:0', dtype=torch.float64)
